In [77]:
%logstop
%logstart -ortq ~/.logs/ml.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [78]:
from static_grader import grader

# ML Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to create effective machine learning models for making predictions. We will be working with nursing home inspection data from the United States, predicting which providers may be fined and for how much.

## Scoring

In this miniproject you will often submit your model's `predict` or `predict_proba` method to the grader. The grader will assess the performance of your model using a scoring metric, comparing it against the score of a reference model. We will use the [average precision score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html). If your model performs better than the reference solution, then you can score higher than 1.0.

**Note:** If you use an estimator that relies on random draws (like a `RandomForestClassifier`) you should set the `random_state=` to an integer so that your results are reproducible. 

## Downloading the data

We can download the data set from Amazon S3:

In [4]:
%%bash
mkdir data
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/providers-train.csv -nc -P ./ml-data
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/providers-metadata.csv -nc -P ./ml-data

mkdir: cannot create directory ‘data’: File exists
--2020-03-08 16:47:20--  http://dataincubator-wqu.s3.amazonaws.com/mldata/providers-train.csv
Resolving dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)... 52.216.85.155
Connecting to dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)|52.216.85.155|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3398120 (3.2M) [text/csv]
Saving to: ‘./ml-data/providers-train.csv’

     0K .......... .......... .......... .......... ..........  1% 59.5M 0s
    50K .......... .......... .......... .......... ..........  3% 49.5M 0s
   100K .......... .......... .......... .......... ..........  4% 44.3M 0s
   150K .......... .......... .......... .......... ..........  6% 48.2M 0s
   200K .......... .......... .......... .......... ..........  7% 44.2M 0s
   250K .......... .......... .......... .......... ..........  9% 48.8M 0s
   300K .......... .......... .......... .......... .....

We'll load the data into a Pandas DataFrame. Several columns will become target labels in future questions. Let's pop those columns out from the data, and drop related columns that are neither targets nor reasonable features (i.e. we don't wouldn't know how many times a facility denied payment before knowing whether it was fined).

The data has many columns. We have also provided a data dictionary.

In [5]:
import numpy as np
import pandas as pd

In [6]:
metadata = pd.read_csv('./ml-data/providers-metadata.csv')
metadata.head()

,Variable,Label,Description,Format
0,PROVNUM,Federal Provider Number,Federal Provider Number,6 alphanumeric characters
1,PROVNAME,Provider Name,Provider Name,text
2,ADDRESS,Provider Address,Provider Address,text
3,CITY,Provider City,Provider City,text
4,STATE,Provider State,Provider State,2-character postal abbreviation


In [7]:
data = pd.read_csv('./ml-data/providers-train.csv', encoding='latin1')

fine_counts = data.pop('FINE_CNT')
fine_totals = data.pop('FINE_TOT')
cycle_2_score = data.pop('CYCLE_2_TOTAL_SCORE')

## Question 1: state_model

A federal agency, Centers for Medicare and Medicaid Services (CMS), imposes regulations on nursing homes. However, nursing homes are inspected by state agencies for compliance with regulations, and fines for violations can vary widely between states.

Let's develop a very simple initial model to predict the amount of fines a nursing home might expect to pay based on its location. Fill in the class definition of the custom estimator, `StateMeanEstimator`, below.

In [8]:
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin

class GroupMeanEstimator(BaseEstimator, RegressorMixin):
    def __init__(self, grouper):
        self.grouper = grouper
        self.group_averages = {}

    def fit(self, X, y):
        # Use self.group_averages to store the average penalty by group
        return self

    def predict(self, X):
        # Return a list of predicted penalties based on group of samples in X
        return [0] * len(X)

After filling in class definition, we can create an instance of the estimator and fit it to the data.

In [23]:
# Solving
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin

class GroupMeanEstimator(BaseEstimator, RegressorMixin):
    def __init__(self, grouper):
        self.grouper = grouper
        self.group_averages = {}

#     def fit(self, X, y):
#         # Use self.group_averages to store the average penalty by group
#         df = pd.concat([X, y], axis=1).groupby(self.grouper)[y.name].mean()
#         #self.unknown_avg = df.mean()
#         self.unknown_avg = y.mean() # thanks ycoulibaly869
#         self.group_averages = df.to_dict()
#         return self
    
    def fit(self, X, y):
        # Use self.group_averages to store the average penalty by group
        self.group_averages = y.groupby(X[self.grouper]).mean().to_dict()
        self.unknown_avg = y.mean()
        return self

#     def predict(self, X):
#         Xt = []
#         if not isinstance(X, pd.DataFrame):
#             X = pd.DataFrame(X)
#         for x in X[self.grouper]:
#             Xt.append(self.group_averages.get(x, self.unknown_avg))
#         # Return a list of predicted penalties based on group of samples in X
#         return np.array(Xt)
    
    def predict(self, X):
        if not isinstance(X, pd.DataFrame):  
            X = pd.DataFrame(X)
        # Return a list of predicted penalties based on group of samples in X
        return np.array([self.group_averages.get(x, self.unknown_avg) for x in X[self.grouper]])
    

In [24]:
from sklearn.pipeline import Pipeline

state_model = Pipeline([
    ('sme', GroupMeanEstimator(grouper='STATE'))
    ])
state_model.fit(data, fine_totals)

Pipeline(memory=None, steps=[('sme', GroupMeanEstimator(grouper='STATE'))],
         verbose=False)

Next we should test that our predict method works.

In [25]:
state_model.predict(data.sample(5))

array([ 5611.81927711, 32656.31538462, 30445.04050633, 21437.39746835,
        2213.51526032])

However, what if we have data from a nursing home in a state (or territory) of the US which is not in the training data?

In [26]:
state_model.predict(pd.DataFrame([{'STATE': 'AS'}]))

array([14969.85768788])

Make sure your model can handle this possibility before submitting your model's predict method to the grader.

In [27]:
grader.score.ml__state_model(state_model.predict)

Your score:  0.9999999999999999


## Question 2: simple_features_model

Nursing homes vary greatly in their business characteristics. Some are owned by the government or non-profits while others are run for profit. Some house a few dozen residents while others house hundreds. Some are located within hospitals and may work with more vulnerable populations. We will try to predict which facilities are fined based on their business characteristics.

We'll begin with columns in our DataFrame containing numeric and boolean features. Some of the rows contain null values; estimators cannot handle null values so these must be imputed or dropped. We will create a `Pipeline` containing transformers that process these features, followed by an estimator.

**Note:** When the grader checks your answer, it passes a list of dictionaries to the `predict` or `predict_proba` method of your estimator, not a DataFrame. This means that your model must work with both data types. For this reason, we've provided a custom `ColumnSelectTransformer` for you to use instead `scikit-learn`'s own `ColumnTransformer`.

In [34]:
from sklearn.impute import SimpleImputer

simple_cols = ['BEDCERT', 'RESTOT', 'INHOSP', 'CCRC_FACIL', 'SFF', 'CHOW_LAST_12MOS', 'SPRINKLER_STATUS', 'EXP_TOTAL', 'ADJ_TOTAL']

class ColumnSelectTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        return X[self.columns]
        
simple_features = Pipeline([
    ('cst', ColumnSelectTransformer(simple_cols)),
])



**Note:** The assertion below assumes the output of `noncategorical_features.fit_transform` is a `ndarray`, not a `DataFrame`.)

In [35]:
assert data['RESTOT'].isnull().sum() > 0
assert not np.isnan(simple_features.fit_transform(data)).any()

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

Now combine the `simple_features` pipeline with an estimator in a new pipeline. Fit `simple_features_model` to the data and submit `simple_features_model.predict_proba` to the grader. You may wish to use cross-validation to tune the hyperparameters of your model.

In [36]:
# simple_features_model = Pipeline([
#     ('simple', simple_features),
#     # add your estimator here
# ])
simple_features = Pipeline([
    ('cst', ColumnSelectTransformer(simple_cols)),
    ('fillna', SimpleImputer(missing_values=np.nan, strategy='mean'))
])


In [37]:
simple_features_model.fit(data, fine_counts > 0)

Pipeline(memory=None,
         steps=[('simple',
                 Pipeline(memory=None,
                          steps=[('cst',
                                  ColumnSelectTransformer(columns=['BEDCERT',
                                                                   'RESTOT',
                                                                   'INHOSP',
                                                                   'CCRC_FACIL',
                                                                   'SFF',
                                                                   'CHOW_LAST_12MOS',
                                                                   'SPRINKLER_STATUS',
                                                                   'EXP_TOTAL',
                                                                   'ADJ_TOTAL'])),
                                 ('fillna',
                                  SimpleImputer(add_indicator=False, copy=True,
                        

In [56]:
### Solving 0.62

from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

simple_cols =['BEDCERT','RESTOT','INHOSP','CCRC_FACIL','SFF','CHOW_LAST_12MOS','SPRINKLER_STATUS','EXP_TOTAL','ADJ_TOTAL']

class ColumnSelectTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        return X[self.columns].to_numpy()


In [75]:
simple_features = Pipeline([
    ('cst', ColumnSelectTransformer(simple_cols)),
    ('fillna', SimpleImputer(missing_values=np.nan, strategy='mean'))
])


simple_features_model = Pipeline([
    ('simple', simple_features),
    ('predictor', RandomForestClassifier(n_estimators=50, max_depth=7.4))
    # add your estimator here
])

simple_features_model.fit(data, fine_counts > 0)

Pipeline(memory=None,
         steps=[('simple',
                 Pipeline(memory=None,
                          steps=[('cst',
                                  ColumnSelectTransformer(columns=['BEDCERT',
                                                                   'RESTOT',
                                                                   'INHOSP',
                                                                   'CCRC_FACIL',
                                                                   'SFF',
                                                                   'CHOW_LAST_12MOS',
                                                                   'SPRINKLER_STATUS',
                                                                   'EXP_TOTAL',
                                                                   'ADJ_TOTAL'])),
                                 ('fillna',
                                  SimpleImputer(add_indicator=False, copy=True,
                        

In [79]:
def positive_probability(model):
    def predict_proba(X):
        return model.predict_proba(X)[:, 1]
    return predict_proba

grader.score.ml__simple_features(positive_probability(simple_features_model))

Your score:  1.0455722144320991


## Question 3: categorical_features

The `'OWNERSHIP'` and `'CERTIFICATION'` columns contain categorical data. We will have to encode the categorical data into numerical features before we pass them to an estimator. Construct one or more pipelines for this purpose. Transformers such as [LabelEncoder](https://scikit-learn.org/0.19/modules/generated/sklearn.preprocessing.LabelEncoder.html#sklearn.preprocessing.LabelEncoder) and [OneHotEncoder](https://scikit-learn.org/0.19/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder) may be useful, but you may also want to define your own transformers.

If you used more than one `Pipeline`, combine them with a `FeatureUnion`. As in Question 2, we will combine this with an estimator, fit it, and submit the `predict_proba` method to the grader.

In [103]:
# from sklearn.pipeline import FeatureUnion

# owner_onehot = Pipeline([
#     ('cst', ColumnSelectTransformer(['OWNERSHIP'])),
# ])

# cert_onehot = Pipeline([
#     ('cst', ColumnSelectTransformer(['CERTIFICATION'])),
# ])

# categorical_features = FeatureUnion([
# ])

from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder

from sklearn.naive_bayes import MultinomialNB

from sklearn.pipeline import FeatureUnion

le = LabelEncoder()

ohe = OneHotEncoder(categorical_features=None, categories=None, drop=None, sparse=False, handle_unknown='error')
# OneHotEncoder(categorical_features=None, categories=None, drop=None, dtype=<class 'numpy.float64'>, handle_unknown='error', n_values=None, sparse=False)


owner_onehot = Pipeline([
    ('cst', ColumnSelectTransformer(['OWNERSHIP'])), ('mt', ohe)
])

cert_onehot = Pipeline([
    ('cst', ColumnSelectTransformer(['CERTIFICATION'])), ('mt', ohe)
])

categorical_features = FeatureUnion([('owner', owner_onehot), ('cert', cert_onehot)
])


rfmodel = RandomForestClassifier(n_estimators=100,max_depth=10)

scaler = StandardScaler(with_mean=False)

mnbmodel = MultinomialNB()




In [101]:
OneHotEncoder?

In [104]:
assert categorical_features.fit_transform(data).shape[0] == data.shape[0]
assert categorical_features.fit_transform(data).dtype == np.float64
assert not np.isnan(categorical_features.fit_transform(data)).any()

As in the previous question, create a model using the `categorical_features`, fit it to the data, and submit its `predict_proba` method to the grader.

In [85]:
# categorical_features_model = Pipeline([
#     ('categorical', categorical_features),
#     # add your estimator here
# ])

categorical_features_model = Pipeline([
    ('categorical', categorical_features),
    ('scaler', scaler),
    ('predictor', mnbmodel)
])

In [105]:
categorical_features_model.fit(data, fine_counts > 0)

Pipeline(memory=None,
         steps=[('categorical',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('owner',
                                                 Pipeline(memory=None,
                                                          steps=[('cst',
                                                                  ColumnSelectTransformer(columns=['OWNERSHIP'])),
                                                                 ('mt',
                                                                  OneHotEncoder(categorical_features=None,
                                                                                categories=None,
                                                                                drop=None,
                                                                                dtype=<class 'numpy.float64'>,
                                                                                handle_unknown='error',
    

In [106]:
grader.score.ml__categorical_features(positive_probability(categorical_features_model))

ValueError: Found unknown categories ['Government - City/county', 'Government - State', 'Government - Federal', 'For profit - Limited Liability company', 'For profit - Individual', 'Government - Hospital district', 'Non profit - Corporation', 'For profit - Corporation', 'Non profit - Church related', 'Non profit - Other', 'Government - City', 'For profit - Partnership', 'Government - County'] in column 0 during transform

In [107]:
### Solving
from sklearn.pipeline import FeatureUnion 
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder 
transformer= OneHotEncoder() 
enc = OneHotEncoder()

owner_onehot = Pipeline([ ('cst', ColumnSelectTransformer(['OWNERSHIP'])), ('enc', enc)

])

cert_onehot = Pipeline([ ('cst', ColumnSelectTransformer(['CERTIFICATION'])), ('tsf', transformer) ])

categorical_features = FeatureUnion([ ('owner',owner_onehot), ('cert', cert_onehot) ])

assert categorical_features.fit_transform(data).shape[0] == data.shape[0] 
assert categorical_features.fit_transform(data).dtype == np.float64 
assert not np.isnan(categorical_features.fit_transform(data).toarray()).any()

from sklearn.naive_bayes import MultinomialNB 
from sklearn.preprocessing import StandardScaler

clf = MultinomialNB(alpha=0.01) 
categorical_features_model = Pipeline([ ('categorical', categorical_features), 
                                       # add your estimator here 
                                       ('scaler', StandardScaler(with_mean=False)), ('predictor', clf) ])

categorical_features_model.fit(data, fine_counts > 0) 
grader.score.ml__categorical_features(positive_probability(categorical_features_model))

Your score:  1.0043781272337378


## Question 4: business_model

Finally, we'll combine `simple_features` and `categorical_features` in a `FeatureUnion`, followed by an estimator in a `Pipeline`. You may want to optimize the hyperparameters of your estimator using cross-validation or try engineering new features (e.g. see [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html)). When you've assembled and trained your model, pass the `predict_proba` method to the grader.

In [ ]:
business_features = FeatureUnion([
    ('simple', simple_features),
    ('categorical', categorical_features)
])

In [ ]:

business_model = Pipeline([
    ('features', business_features),
    # add your estimator here
])

In [ ]:
business_model.fit(data, fine_counts > 0)

In [112]:
# Solving
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression

business_features = FeatureUnion([
('simple', simple_features),
('categorical', categorical_features) ])

business_model = Pipeline([
('features', business_features),
('poly', PolynomialFeatures(2)),
('lr', LogisticRegression())])
# ('lr',LogisticRegression(multi_class='multinomial',solver ='newton-cg'))])
business_model.fit(data, fine_counts > 0)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('simple',
                                                 Pipeline(memory=None,
                                                          steps=[('cst',
                                                                  ColumnSelectTransformer(columns=['BEDCERT',
                                                                                                   'RESTOT',
                                                                                                   'INHOSP',
                                                                                                   'CCRC_FACIL',
                                                                                                   'SFF',
                                                                                                   'CHOW_LAST_12MOS',
                                      

In [113]:
grader.score.ml__business_model(positive_probability(business_model))

Your score:  0.9914203514059667


## Question 5: survey_results

Surveys reveal safety and health deficiencies at nursing homes that may indicate risk for incidents (and penalties). CMS routinely makes surveys of nursing homes. Build a model that combines the `business_features` of each facility with its cycle 1 survey results, as well as the time between the cycle 1 and cycle 2 survey to predict the cycle 2 total score.

First, let's create a transformer to calculate the difference in time between the cycle 1 and cycle 2 surveys.

In [ ]:
# class TimedeltaTransformer(BaseEstimator, TransformerMixin):
#     def __init__(self, t1_col, t2_col):
#         self.t1_col = t1_col
#         self.t2_col = t2_col

#     def fit(self, X, y=None):
#         return self

#     def transform(self, X):

In [114]:
class TimedeltaTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, t1_col, t2_col):
        self.t1_col = t1_col
        self.t2_col = t2_col

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        return np.array((pd.to_datetime(X[self.t1_col]) - pd.to_datetime(X[self.t2_col])).dt.days).reshape(-1,1)

In [115]:
cycle_1_date = 'CYCLE_1_SURVEY_DATE'
cycle_2_date = 'CYCLE_2_SURVEY_DATE'
time_feature = TimedeltaTransformer(cycle_1_date, cycle_2_date)

In the cell below we'll collect the cycle 1 survey features.

In [120]:
cycle_1_cols = ['CYCLE_1_DEFS', 'CYCLE_1_NFROMDEFS', 'CYCLE_1_NFROMCOMP',
                'CYCLE_1_DEFS_SCORE', 'CYCLE_1_NUMREVIS',
                'CYCLE_1_REVISIT_SCORE', 'CYCLE_1_TOTAL_SCORE']
cycle_1_features = ColumnSelectTransformer(cycle_1_cols)

In [121]:
# from sklearn.ensemble import RandomForestRegressor

# survey_model = Pipeline([
#     ('features', FeatureUnion([
#         ('business', business_features),
#         ('survey', cycle_1_features),
#         ('time', time_feature)
#     ])),
#     # add your estimator here
# ])

# from sklearn.ensemble import RandomForestRegressor


# survey_model = Pipeline([
#     ('features', FeatureUnion([
#         ('business', business_features),
#         ('survey', cycle_1_features),
#         ('time', time_feature)
#     ])),
#     # add your estimator here
#     ('lr' , RandomForestRegressor(n_estimators = 50,
#                                      max_depth = 5))
# ])

## SOlving
from sklearn.linear_model import Lasso
survey_model = Pipeline([
    ('features', FeatureUnion([
        ('business', business_features),
        ('survey', cycle_1_features),
        ('time', time_feature)
    ])),
    # add your estimator here
    ('lr' , Lasso(alpha= 1.6, max_iter=1000))
])


### Another method - 1.208
# clf = Lasso(alpha= 51)
# survey_model = Pipeline([
#     ('features', FeatureUnion([
#         ('business', business_features),
#         ('survey', cycle_1_features),
#         ('time', time_feature)
#     ])),
#     # add your estimator here
#     #('scale', RobustScaler(with_centering=False)),
#     #('norm', Normalizer(norm='l2')),
#     ('poly', PolynomialFeatures(2)),
#     ('red', TruncatedSVD(49)),
#     #('reduce', NMF(n_components=24)),
#     ('regressor', clf)
# ])

In [122]:
survey_model.fit(data, cycle_2_score.astype(int))

Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('business',
                                                 FeatureUnion(n_jobs=None,
                                                              transformer_list=[('simple',
                                                                                 Pipeline(memory=None,
                                                                                          steps=[('cst',
                                                                                                  ColumnSelectTransformer(columns=['BEDCERT',
                                                                                                                                   'RESTOT',
                                                                                                                                   'INHOSP',
                                               

In [123]:
grader.score.ml__survey_model(survey_model.predict)

Your score:  1.0978317813401475


In [ ]:
### Suggestion
bs = FeatureUnion([
    ('business', business_features),
    ('survey', cycle_1_features),
    ('time', time_feature)
    ])

lg = Pipeline([
    ('bs', bs),
    ('scale', RobustScaler(with_centering=False)),
    ('norm', Normalizer(norm='l2')),
    ('lg', EstTransformer(LogisticRegression(solver='liblinear', C=0.1, max_iter=10000, multi_class='auto')))
])
#the score gets to 1.198

# Add RandomForest:

rf = Pipeline([
    ('bs', bs),
    ('norm', Normalizer(norm='l2')),
    ('rf', EstTransformer(RandomForestClassifier(n_estimators=64, max_depth=10, min_samples_split=5)))
])


ensemble = FeatureUnion([
    ('rf',rf),
    ('lg', lg),
])
# and put everything together

clf = Lasso(alpha=1.6)
survey_model = Pipeline([
    ('features', FeatureUnion([
    ('business', business_features),
    ('survey', cycle_1_features),
    ('time', time_feature),
    ('new',ensemble)
    ])),
    # add your estimator here
    ('regressor', clf)
])

# EstTransformer, taken from the video:

from sklearn.base import BaseEstimator, TransformerMixin

class EstTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, estimator):
        self.estimator = estimator

    def fit(self, X, y):
        self.estimator.fit(X, y)
        return self

    def transform(self, X):
        return np.atleast_2d(self.estimator.predict(X)).T

*Copyright &copy; 2019 The Data Incubator.  All rights reserved.*